In [1]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import KFold

from util import gini_normalized
from parameters import parameters, batch_size, epochs, layers, activation_functions, loss, alpha
from preprocessing import preproc

In [2]:
# Importing the train dataset
dataset_train = pd.read_csv('train.csv')

# Importing the test dataset
dataset_test = pd.read_csv('test.csv')
# preprocessing train dataset
X_train, y_train = preproc(dataset_train, 'train', oneHot=False, feature_selection=True)

# preprocessing test dataset
X_test, y_test = preproc(dataset_test, 'test', oneHot=False, feature_selection=True)

getting feature selection from cache
replacing missing values
number of examples: 595212
getting feature selection from cache
replacing missing values
number of examples: 892816


In [3]:
# Now let's make the Classifier!
# Fitting Random Forest Classification to the Training set

class_weight = {0: 1., 1: alpha}
K = 5
kf = KFold(n_splits=K, random_state=42, shuffle=True)

In [4]:
i=0
results = []
for train_index, test_index in kf.split(X_train):
    train_x, train_y = X_train[train_index], y_train[train_index]
    eval_x, eval_y = X_train[test_index], y_train[test_index]
    classifier = RandomForestClassifier(n_estimators=30, criterion = 'gini', max_depth=5, random_state = 1, max_features='auto', class_weight=class_weight)
    classifier.fit(train_x, train_y)
    res_eval = classifier.predict(eval_x)
    res = classifier.predict(X_test)
    results.append(res)
    print('gini_eval', i)
    gini_score = gini_normalized(eval_y, res_eval)
    print(gini_score)
    i+=1


gini_eval 0
0.0396813383823
gini_eval 1
0.0437326368221
gini_eval 2
0.0502027668516
gini_eval 3
0.0404943812147
gini_eval 4
0.0304418354861


In [5]:
def to_csv(y_pred, ids):
    import csv
    with open('sumbission_5Kfold_random_forest.csv', 'w') as csvfile:
        spamwriter = csv.writer(csvfile, delimiter=',')
        spamwriter.writerow(['id', 'target'])
        for i in range(len(y_pred)):
            spamwriter.writerow([ids[i], y_pred[i]])

In [6]:
submission = (results[0] + results[1] + results[2] + results[3] + results[4]) / 5
idx = dataset_test.iloc[:, 0].values
to_csv(submission,idx)

gini score for this submission : 0.04576